## PM Check-in Data Prep

In [9]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns
import numpy as np

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()


UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.0.0) is available. Your version (1.9.4) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.0.0 release notes

https://python-docs.synapse.org/build/html/news.html



Welcome, Sean Deering!



### Get Raw PM-Checkin Data

In [10]:
pmcheckin_raw_id = 'syn7117927'
pmcheckin = pd.read_csv(syn.get(pmcheckin_raw_id).path,sep='\t',error_bad_lines=False)

Skipping line 18432: expected 18 fields, saw 22



### Replace white space with NaN

In [11]:
pmcheckin = pmcheckin.replace(r'^\s*$', np.nan, regex=True)

### Process Raw PM-Checkin data

In [12]:
#rename these variables for consistency
pmcheckin = pmcheckin.rename(columns = 
                             { 'PMCH-1':'PMCH1',
                              'PMCH-2a':'PMCH2A',
                              'PMCH-3':'PMCH3'})

In [13]:
#remove brackets
def remove_brackets(col):
    return( col.astype(str).str.replace(']', '').str.replace('[',''))
pmcheckin.NapCount=remove_brackets(pmcheckin.NapCount)
pmcheckin.PMCH1=remove_brackets(pmcheckin.PMCH1)
pmcheckin.PMCH3=remove_brackets(pmcheckin.PMCH3)

#recode variables for PMCH-1 into what they were supposed to be
pmcheckin['PMCH1'].replace('Very poor',1,inplace=True)
pmcheckin['PMCH1'].replace('Poor',2,inplace=True)
pmcheckin['PMCH1'].replace('Fair',3,inplace=True)

### Filter based on age

In [ ]:
underage_participants = pd.read_csv( syn.get('syn21905452').path, sep="\t")
pmcheckin = pmcheckin[~pmcheckin.participantId.isin(underage_participants.participantId)]

### Fix Alcohol Related Columns

In [14]:
alcohol_related_cols = ['alcohol_only', 'alcohol&medication_alcohol', 'caffeine&alcohol_alcohol', 'caffeine&alcohol&medication_alcohol']
alcohol = pmcheckin[alcohol_related_cols]
alcohol = alcohol.sum(axis=1, skipna=True).astype(int)
alcohol.quantile([.1, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999])
alcohol.describe()

## If the #alcohol drinks is > .99 percentile of the sample in this case > 7 , we list that value is as 
alcohol[ alcohol > 7 ] = 'DATA QUALITY ISSUE'
alcohol.value_counts()

pmcheckin['alcohol'] = alcohol

0.100     0.000
0.500     0.000
0.600     0.000
0.800     0.000
0.850     1.000
0.900     2.000
0.950     3.000
0.970     4.000
0.990     7.000
0.999    25.338
dtype: float64

count    4.366300e+04
mean     2.827497e+05
std      5.908241e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.234568e+10
dtype: float64

0                     35947
1                      3008
2                      2159
3                      1150
4                       487
DATA QUALITY ISSUE      396
5                       311
6                       163
7                        42
dtype: int64

### Fix Caffeine Related Columns

In [15]:
caffeine_related_cols = ['caffeine_only','caffeine&medication_caffeine','caffeine&alcohol_caffeine','caffeine&alcohol&medication_caffeine' ]
caffeine = pmcheckin[caffeine_related_cols]

caffeine = caffeine.sum(axis=1, skipna=True).astype(int)
caffeine.quantile([.1, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])
caffeine.describe()

## If the #caffeine drinks is > .99 percentile of the sample in this case > 32 , we list that value is as 
caffeine[ caffeine > 32 ] = 'DATA QUALITY ISSUE'
caffeine.value_counts()

pmcheckin['caffeine'] = caffeine

0.100    0.000000e+00
0.500    2.000000e+00
0.600    2.000000e+00
0.800    4.000000e+00
0.850    6.000000e+00
0.900    1.000000e+01
0.950    1.600000e+01
0.970    2.400000e+01
0.990    3.200000e+01
0.999    6.400000e+01
1.000    3.001827e+10
dtype: float64

count    4.366300e+04
mean     6.875029e+05
std      1.436577e+08
min      0.000000e+00
25%      0.000000e+00
50%      2.000000e+00
75%      4.000000e+00
max      3.001827e+10
dtype: float64

0                     12545
2                      8242
1                      5976
3                      5107
4                      3471
12                     1420
6                      1299
8                      1180
5                      1172
24                      680
16                      660
20                      444
DATA QUALITY ISSUE      428
10                      256
32                      160
7                       155
18                      106
30                       55
28                       54
14                       48
9                        39
25                       36
26                       36
15                       34
22                       23
11                       11
21                        8
13                        8
19                        4
17                        3
27                        2
23                        1
dtype: int64

### Drop Older Alcohol and Caffeine related cols 

In [16]:
alcohol_related_cols.extend(caffeine_related_cols)
removeCols = list(set(alcohol_related_cols))
pmcheckin = pmcheckin.drop(removeCols, axis=1)

### Clean PMCH2A (Nap Lengths)

In [17]:
pmcheckin.PMCH2A.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])
pmcheckin.PMCH2A.describe()

0.010         1.0
0.050         2.0
0.500        30.0
0.600        45.0
0.800        90.0
0.850       120.0
0.900       120.0
0.950       180.0
0.970       218.0
0.990       300.0
0.999       600.0
1.000    207888.0
Name: PMCH2A, dtype: float64

count     11728.000000
mean         72.309430
std        1920.520248
min           0.000000
25%          15.000000
50%          30.000000
75%          65.000000
max      207888.000000
Name: PMCH2A, dtype: float64

In [19]:
## If the nap duration is > .99 percentile of the sample in this case > 300 , we list that value is as 
pmcheckin.PMCH2A[ pmcheckin.PMCH2A > 300 ] = 'DATA QUALITY ISSUE'
pmcheckin.PMCH2A.value_counts()

[WARNING] /usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  



30.0                  1220
60.0                  1210
20.0                   973
15.0                   865
10.0                   854
120.0                  764
5.0                    737
45.0                   657
90.0                   590
2.0                    341
1.0                    339
180.0                  303
25.0                   219
35.0                   207
40.0                   203
3.0                    200
75.0                   161
240.0                  154
150.0                  125
50.0                   122
0.0                    108
DATA QUALITY ISSUE     102
70.0                    78
80.0                    67
300.0                   61
100.0                   60
65.0                    60
4.0                     58
55.0                    39
12.0                    37
                      ... 
152.0                    1
174.0                    1
129.0                    1
278.0                    1
133.0                    1
172.0                    1
1

## Create External Copy of the data

In [22]:
#download the data 
sharing_info = pd.read_excel(syn.get('syn21557215').path)
healthCodes_with_broadsharing = sharing_info[sharing_info.sharing == 'all_qualified_researchers']

EXTERNAL_PMCHECKIN_DATA = pmcheckin[pmcheckin.participantId.isin(healthCodes_with_broadsharing['participant id'])]
EXTERNAL_PMCHECKIN_DATA.shape

#drop columns with free text fields
EXTERNAL_PMCHECKIN_DATA = EXTERNAL_PMCHECKIN_DATA.drop(columns=['medication_only','caffeine&medication_medication',
                                                                'alcohol&medication_medication',
                                                                'caffeine&alcohol&medication_medication'], axis=1)
EXTERNAL_PMCHECKIN_DATA.shape

(27340, 12)

(27340, 8)

### External - Upload to Synapse

In [23]:
SH_EXTERNAL_PROJECT = 'syn18492837'
table_schema_external = Schema(name='PM Check-in',
                               columns=as_table_columns(EXTERNAL_PMCHECKIN_DATA),
                               parent=SH_EXTERNAL_PROJECT)
pmCheckin_synTable_external = syn.store(Table(table_schema_external,EXTERNAL_PMCHECKIN_DATA))

4295

### Internal - Upload to Synapse

In [24]:
SH_INTERNAL_PROJECT = 'syn7066726'
table_schema_internal = Schema(name='PM Check-in Internal',
                               columns=as_table_columns(pmcheckin),
                               parent=SH_INTERNAL_PROJECT)
pmCheckin_synTable_internal = syn.store(Table(table_schema_internal,pmcheckin))

In [25]:
activity=Activity(name= 'PM Check-in', 
                  description='Process and convert raw data to table format', 
                  used=pmcheckin_raw_id, 
                  executed='https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_PM_Checkin.ipynb')
#Internal 
syn.setProvenance(pmCheckin_synTable_internal, activity)

#External
syn.setProvenance(pmCheckin_synTable_external, activity)

{u'createdBy': u'3334346',
 u'createdOn': u'2020-04-08T16:32:37.010Z',
 u'description': u'Process and convert raw data to table format',
 u'etag': u'eb9760c8-6c95-4e5d-87bf-866da865486c',
 u'id': u'10232181',
 u'modifiedBy': u'3334346',
 u'modifiedOn': u'2020-04-08T16:32:37.010Z',
 u'name': u'PM Check-in',
 u'used': [{u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedURL',
   u'name': u'https://github.com/sdeering/SLEEPHEALTHv2-Data-Release/Create_PM_Checkin.ipynb',
   u'url': u'https://github.com/sdeering/SLEEPHEALTHv2-Data-Release/Create_PM_Checkin.ipynb',
   u'wasExecuted': True},
  {u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedEntity',
   u'reference': {u'targetId': u'syn7117927', u'targetVersionNumber': 784},
   u'wasExecuted': False}]}

{u'createdBy': u'3334346',
 u'createdOn': u'2020-04-08T16:32:56.267Z',
 u'description': u'Process and convert raw data to table format',
 u'etag': u'3724008e-15e3-4eb4-91cb-98e8cdcbfe51',
 u'id': u'10232182',
 u'modifiedBy': u'3334346',
 u'modifiedOn': u'2020-04-08T16:32:56.267Z',
 u'name': u'PM Check-in',
 u'used': [{u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedURL',
   u'name': u'https://github.com/sdeering/SLEEPHEALTHv2-Data-Release/Create_PM_Checkin.ipynb',
   u'url': u'https://github.com/sdeering/SLEEPHEALTHv2-Data-Release/Create_PM_Checkin.ipynb',
   u'wasExecuted': True},
  {u'concreteType': u'org.sagebionetworks.repo.model.provenance.UsedEntity',
   u'reference': {u'targetId': u'syn7117927', u'targetVersionNumber': 784},
   u'wasExecuted': False}]}